In [1]:
import numpy as np
import pandas as pd
import random

from sklearn.ensemble import RandomForestClassifier
from src.utils import preprocess
from tqdm import tqdm

from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense
from keras.layers import Embedding, LSTM
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import losses
from keras import regularizers
from keras.constraints import max_norm
from keras.wrappers.scikit_learn import KerasClassifier

import time

from sklearn.model_selection import StratifiedKFold

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Loading data

In [2]:
Data_X_train = pd.read_csv('data/challenge_fichier_dentrees_dentrainement_challenge_nba/train.csv')
Data_Y_train = pd.read_csv('data/challenge_fichier_de_sortie_dentrainement_challenge_nba.csv', sep=';')

In [3]:
X = Data_X_train.as_matrix()[:,1:]
Y = Data_Y_train.as_matrix()[:,1:]

# K fold functions

In [4]:
def train_and_evaluate(model, X_train, Y_train, X_val, Y_val, epochs, batch_size, verbose):
    training = model.fit(X_train, Y_train, epochs = epochs, batch_size = batch_size, verbose = verbose)
    evalu = model.evaluate(X_val, Y_val)
    
    return training.history['acc'][-1], evalu[1]

In [5]:
skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 7)

In [12]:
def kfold_print(skf_splitter, X, Y, create_model_func, epochs, batch_size, verbose):
    accs_train = []
    accs_val = []
    for train, val in skf.split(X, Y):
        model = create_model_func()
        Y_train = np_utils.to_categorical(Y[train], 2)
        Y_val = np_utils.to_categorical(Y[val], 2)
        acc_train, acc_val = train_and_evaluate(model, 
                                                X[train], Y_train, X[val], Y_val, 
                                                epochs = epochs, batch_size = batch_size, verbose = verbose)

        accs_train.append(acc_train)
        accs_val.append(acc_val)
        print('(Training, Validation) accuracies: ({0:.2f},{1:.2f})'.format(100*acc_train, 100*acc_val))

    print('Mean Training Accuracy: {0:.2f} +/- {1:.2f}'.format(100*np.mean(accs_train), 100*np.std(accs_train)))
    print('Mean Validation Accuracy: {0:.2f} +/- {1:.2f}'.format(100*np.mean(accs_val), 100*np.std(accs_val)))

# 15sec subsequences - Aggregation = Mean

In [39]:
X_train = X.reshape((len(X), 11, 15, -1), order = 'F')
X_train = X_train.mean(axis = 2)

In [13]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(30, 
                        input_shape = (11, 96), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 20,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 0s 139us/step
(Training, Validation) accuracies: (69.99,72.02)
2516/2516 [==============================] - 0s 151us/step
(Training, Validation) accuracies: (70.03,70.79)
2515/2515 [==============================] - 0s 159us/step
(Training, Validation) accuracies: (70.49,69.98)
2515/2515 [==============================] - 0s 174us/step
(Training, Validation) accuracies: (70.77,70.97)
2514/2514 [==============================] - 0s 185us/step
(Training, Validation) accuracies: (71.13,68.10)
Mean Training Accuracy: 70.48 +/- 0.44
Mean Validation Accuracy: 70.37 +/- 1.31


In [14]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(50, 
                        input_shape = (11, 96), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 20,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 0s 196us/step
(Training, Validation) accuracies: (71.10,71.94)
2516/2516 [==============================] - 1s 210us/step
(Training, Validation) accuracies: (71.05,71.74)
2515/2515 [==============================] - 1s 223us/step
(Training, Validation) accuracies: (71.81,70.85)
2515/2515 [==============================] - 1s 234us/step
(Training, Validation) accuracies: (71.52,71.13)
2514/2514 [==============================] - 1s 260us/step
(Training, Validation) accuracies: (71.89,68.93)
Mean Training Accuracy: 71.48 +/- 0.35
Mean Validation Accuracy: 70.92 +/- 1.07


In [15]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(75, 
                        input_shape = (11, 96), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 20,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 1s 268us/step
(Training, Validation) accuracies: (72.95,71.38)
2516/2516 [==============================] - 1s 272us/step
(Training, Validation) accuracies: (72.23,71.50)
2515/2515 [==============================] - 1s 279us/step
(Training, Validation) accuracies: (72.64,69.82)
2515/2515 [==============================] - 1s 297us/step
(Training, Validation) accuracies: (73.03,70.46)
2514/2514 [==============================] - 1s 306us/step
(Training, Validation) accuracies: (73.75,68.30)
Mean Training Accuracy: 72.92 +/- 0.50
Mean Validation Accuracy: 70.29 +/- 1.17


In [16]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(100, 
                        input_shape = (11, 96), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 20,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 1s 326us/step
(Training, Validation) accuracies: (73.15,72.54)
2516/2516 [==============================] - 1s 340us/step
(Training, Validation) accuracies: (73.47,72.10)
2515/2515 [==============================] - 1s 354us/step
(Training, Validation) accuracies: (74.02,69.22)
2515/2515 [==============================] - 1s 366us/step
(Training, Validation) accuracies: (73.57,70.74)
2514/2514 [==============================] - 1s 388us/step
(Training, Validation) accuracies: (74.41,67.34)
Mean Training Accuracy: 73.72 +/- 0.44
Mean Validation Accuracy: 70.39 +/- 1.91


In [17]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(150, 
                        input_shape = (11, 96), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 20,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 1s 383us/step
(Training, Validation) accuracies: (76.67,71.34)
2516/2516 [==============================] - 1s 409us/step
(Training, Validation) accuracies: (75.74,71.03)
2515/2515 [==============================] - 1s 411us/step
(Training, Validation) accuracies: (76.18,69.98)
2515/2515 [==============================] - 1s 435us/step
(Training, Validation) accuracies: (75.58,70.78)
2514/2514 [==============================] - 1s 440us/step
(Training, Validation) accuracies: (76.11,67.42)
Mean Training Accuracy: 76.05 +/- 0.38
Mean Validation Accuracy: 70.11 +/- 1.42


In [40]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(200, 
                        input_shape = (11, 96), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 20,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (79.16,70.39)
2516/2516 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (78.61,70.51)
2515/2515 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (80.02,68.11)
2515/2515 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (78.10,70.06)
2514/2514 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (78.95,67.06)
Mean Training Accuracy: 78.97 +/- 0.64
Mean Validation Accuracy: 69.23 +/- 1.39


In [41]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(300, 
                        input_shape = (11, 96), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 20,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (84.09,67.45)
2516/2516 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (83.55,69.95)
2515/2515 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (83.33,67.91)
2515/2515 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (82.40,68.43)
2514/2514 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (83.68,68.34)
Mean Training Accuracy: 83.41 +/- 0.56
Mean Validation Accuracy: 68.42 +/- 0.84


In [18]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(30, 
                        input_shape = (11, 96), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 1s 468us/step
(Training, Validation) accuracies: (70.37,72.14)
2516/2516 [==============================] - 1s 458us/step
(Training, Validation) accuracies: (70.11,71.07)
2515/2515 [==============================] - 1s 466us/step
(Training, Validation) accuracies: (70.35,69.66)
2515/2515 [==============================] - 1s 487us/step
(Training, Validation) accuracies: (71.11,71.09)
2514/2514 [==============================] - 1s 484us/step
(Training, Validation) accuracies: (71.21,68.50)
Mean Training Accuracy: 70.63 +/- 0.44
Mean Validation Accuracy: 70.49 +/- 1.27


In [19]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(50, 
                        input_shape = (11, 96), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 1s 500us/step
(Training, Validation) accuracies: (71.88,72.22)
2516/2516 [==============================] - 1s 517us/step
(Training, Validation) accuracies: (71.79,71.46)
2515/2515 [==============================] - 1s 569us/step
(Training, Validation) accuracies: (72.61,69.26)
2515/2515 [==============================] - 1s 542us/step
(Training, Validation) accuracies: (72.53,70.66)
2514/2514 [==============================] - 1s 556us/step
(Training, Validation) accuracies: (72.99,67.94)
Mean Training Accuracy: 72.36 +/- 0.46
Mean Validation Accuracy: 70.31 +/- 1.54


In [20]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(75, 
                        input_shape = (11, 96), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 1s 556us/step
(Training, Validation) accuracies: (73.12,70.99)
2516/2516 [==============================] - 1s 572us/step
(Training, Validation) accuracies: (73.79,71.90)
2515/2515 [==============================] - 1s 595us/step
(Training, Validation) accuracies: (74.11,69.30)
2515/2515 [==============================] - 2s 611us/step
(Training, Validation) accuracies: (73.89,70.02)
2514/2514 [==============================] - 2s 617us/step
(Training, Validation) accuracies: (74.18,68.02)
Mean Training Accuracy: 73.82 +/- 0.38
Mean Validation Accuracy: 70.05 +/- 1.34


In [21]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(100, 
                        input_shape = (11, 96), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 2s 655us/step
(Training, Validation) accuracies: (76.18,70.03)
2516/2516 [==============================] - 2s 646us/step
(Training, Validation) accuracies: (76.09,70.47)
2515/2515 [==============================] - 2s 663us/step
(Training, Validation) accuracies: (75.82,68.99)
2515/2515 [==============================] - 2s 651us/step
(Training, Validation) accuracies: (76.49,70.66)
2514/2514 [==============================] - 2s 680us/step
(Training, Validation) accuracies: (77.12,67.18)
Mean Training Accuracy: 76.34 +/- 0.45
Mean Validation Accuracy: 69.47 +/- 1.28


In [22]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(150, 
                        input_shape = (11, 96), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 2s 705us/step
(Training, Validation) accuracies: (79.77,69.59)
2516/2516 [==============================] - 2s 709us/step
(Training, Validation) accuracies: (79.86,70.11)
2515/2515 [==============================] - 2s 725us/step
(Training, Validation) accuracies: (78.80,68.91)
2515/2515 [==============================] - 2s 723us/step
(Training, Validation) accuracies: (80.12,68.59)
2514/2514 [==============================] - 2s 740us/step
(Training, Validation) accuracies: (81.03,67.10)
Mean Training Accuracy: 79.92 +/- 0.71
Mean Validation Accuracy: 68.86 +/- 1.03


In [42]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(200, 
                        input_shape = (11, 96), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (84.28,69.40)
2516/2516 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (83.51,67.33)
2515/2515 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (84.22,69.66)
2515/2515 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (84.70,68.23)
2514/2514 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (83.96,68.06)
Mean Training Accuracy: 84.13 +/- 0.39
Mean Validation Accuracy: 68.54 +/- 0.87


In [43]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(300, 
                        input_shape = (11, 96), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (89.28,68.00)
2516/2516 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (88.89,69.04)
2515/2515 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (89.53,67.44)
2515/2515 [==============================] - 6s 2ms/step
(Training, Validation) accuracies: (89.20,68.83)
2514/2514 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (88.63,66.87)
Mean Training Accuracy: 89.11 +/- 0.32
Mean Validation Accuracy: 68.03 +/- 0.82


# 10sec subsequences - Aggregation = Mean

In [24]:
X_train = X.reshape((len(X), 11, 10, -1), order = 'F')
X_train = X_train.mean(axis = 2)

In [25]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(30, 
                        input_shape = (11, 144), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 20,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 2s 775us/step
(Training, Validation) accuracies: (68.93,69.87)
2516/2516 [==============================] - 2s 765us/step
(Training, Validation) accuracies: (69.41,71.26)
2515/2515 [==============================] - 2s 801us/step
(Training, Validation) accuracies: (69.82,70.46)
2515/2515 [==============================] - 2s 808us/step
(Training, Validation) accuracies: (70.12,70.22)
2514/2514 [==============================] - 2s 817us/step
(Training, Validation) accuracies: (70.93,68.14)
Mean Training Accuracy: 69.84 +/- 0.68
Mean Validation Accuracy: 69.99 +/- 1.03


In [26]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(50, 
                        input_shape = (11, 144), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 20,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 2s 836us/step
(Training, Validation) accuracies: (70.95,72.10)
2516/2516 [==============================] - 2s 858us/step
(Training, Validation) accuracies: (71.34,70.75)
2515/2515 [==============================] - 2s 867us/step
(Training, Validation) accuracies: (71.55,69.74)
2515/2515 [==============================] - 2s 887us/step
(Training, Validation) accuracies: (71.65,70.50)
2514/2514 [==============================] - 2s 897us/step
(Training, Validation) accuracies: (72.53,67.46)
Mean Training Accuracy: 71.61 +/- 0.52
Mean Validation Accuracy: 70.11 +/- 1.53


In [27]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(75, 
                        input_shape = (11, 144), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 20,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 2s 907us/step
(Training, Validation) accuracies: (72.31,71.07)
2516/2516 [==============================] - 2s 919us/step
(Training, Validation) accuracies: (71.47,70.35)
2515/2515 [==============================] - 2s 959us/step
(Training, Validation) accuracies: (73.23,69.82)
2515/2515 [==============================] - 2s 943us/step
(Training, Validation) accuracies: (72.75,70.30)
2514/2514 [==============================] - 2s 970us/step
(Training, Validation) accuracies: (73.50,68.62)
Mean Training Accuracy: 72.65 +/- 0.72
Mean Validation Accuracy: 70.03 +/- 0.81


In [28]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(100, 
                        input_shape = (11, 144), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 20,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 2s 986us/step
(Training, Validation) accuracies: (73.84,71.22)
2516/2516 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (73.34,70.51)
2515/2515 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (73.73,69.78)
2515/2515 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (73.68,69.66)
2514/2514 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (74.13,68.26)
Mean Training Accuracy: 73.74 +/- 0.26
Mean Validation Accuracy: 69.89 +/- 0.99


In [29]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(150, 
                        input_shape = (11, 144), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 20,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (77.04,71.30)
2516/2516 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (75.78,69.91)
2515/2515 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (77.18,68.67)
2515/2515 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (75.16,68.79)
2514/2514 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (77.20,66.35)
Mean Training Accuracy: 76.47 +/- 0.84
Mean Validation Accuracy: 69.00 +/- 1.63


In [37]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(200, 
                        input_shape = (11, 144), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 20,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (79.77,69.99)
2516/2516 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (78.99,70.67)
2515/2515 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (78.54,69.34)
2515/2515 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (78.75,70.10)
2514/2514 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (79.81,68.14)
Mean Training Accuracy: 79.17 +/- 0.52
Mean Validation Accuracy: 69.65 +/- 0.86


In [38]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(300, 
                        input_shape = (11, 144), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 20,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (83.49,69.12)
2516/2516 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (82.03,69.59)
2515/2515 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (83.20,69.74)
2515/2515 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (82.37,68.47)
2514/2514 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (83.75,66.55)
Mean Training Accuracy: 82.97 +/- 0.66
Mean Validation Accuracy: 68.69 +/- 1.16


In [30]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(30, 
                        input_shape = (11, 144), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (69.70,70.43)
2516/2516 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (69.92,70.75)
2515/2515 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (71.33,69.82)
2515/2515 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (70.56,70.10)
2514/2514 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (71.29,68.66)
Mean Training Accuracy: 70.56 +/- 0.67
Mean Validation Accuracy: 69.95 +/- 0.72


In [31]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(50, 
                        input_shape = (11, 144), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (71.40,71.30)
2516/2516 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (71.96,70.55)
2515/2515 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (72.30,69.58)
2515/2515 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (72.34,71.09)
2514/2514 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (72.88,68.14)
Mean Training Accuracy: 72.18 +/- 0.49
Mean Validation Accuracy: 70.13 +/- 1.16


In [32]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(75, 
                        input_shape = (11, 144), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (73.59,71.30)
2516/2516 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (73.43,70.11)
2515/2515 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (73.58,70.18)
2515/2515 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (74.17,70.02)
2514/2514 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (74.54,67.66)
Mean Training Accuracy: 73.86 +/- 0.42
Mean Validation Accuracy: 69.85 +/- 1.19


In [33]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(100, 
                        input_shape = (11, 144), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (75.46,70.71)
2516/2516 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (75.72,70.71)
2515/2515 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (75.76,69.03)
2515/2515 [==============================] - 4s 1ms/step
(Training, Validation) accuracies: (76.07,69.86)
2514/2514 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (75.96,68.06)
Mean Training Accuracy: 75.79 +/- 0.21
Mean Validation Accuracy: 69.67 +/- 1.02


In [34]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(150, 
                        input_shape = (11, 144), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 4s 1ms/step
(Training, Validation) accuracies: (80.01,71.18)
2516/2516 [==============================] - 4s 1ms/step
(Training, Validation) accuracies: (79.48,69.79)
2515/2515 [==============================] - 4s 1ms/step
(Training, Validation) accuracies: (79.73,68.83)
2515/2515 [==============================] - 4s 1ms/step
(Training, Validation) accuracies: (80.03,69.11)
2514/2514 [==============================] - 4s 1ms/step
(Training, Validation) accuracies: (80.15,67.18)
Mean Training Accuracy: 79.88 +/- 0.24
Mean Validation Accuracy: 69.22 +/- 1.30


In [35]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(200, 
                        input_shape = (11, 144), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 4s 1ms/step
(Training, Validation) accuracies: (82.70,69.40)
2516/2516 [==============================] - 4s 1ms/step
(Training, Validation) accuracies: (82.89,70.55)
2515/2515 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (82.82,69.82)
2515/2515 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (82.89,68.59)
2514/2514 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (83.34,66.47)
Mean Training Accuracy: 82.93 +/- 0.22
Mean Validation Accuracy: 68.96 +/- 1.40


In [36]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(300, 
                        input_shape = (11, 144), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (88.42,68.00)
2516/2516 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (88.58,69.24)
2515/2515 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (88.79,68.71)
2515/2515 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (89.23,68.35)
2514/2514 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (89.59,65.35)
Mean Training Accuracy: 88.92 +/- 0.43
Mean Validation Accuracy: 67.93 +/- 1.35


# 20 sec subsequences - Aggregation = Mean

In [7]:
X_train = X.reshape((len(X), 11, 20, -1), order = 'F')
X_train = X_train.mean(axis = 2)

In [13]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(100, 
                        input_shape = (11, 72), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 75,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 0s 133us/step
(Training, Validation) accuracies: (72.68,71.94)
2516/2516 [==============================] - 0s 145us/step
(Training, Validation) accuracies: (72.42,71.82)
2515/2515 [==============================] - 0s 159us/step
(Training, Validation) accuracies: (72.58,69.54)
2515/2515 [==============================] - 0s 169us/step
(Training, Validation) accuracies: (73.09,70.30)
2514/2514 [==============================] - 0s 180us/step
(Training, Validation) accuracies: (73.51,68.62)
Mean Training Accuracy: 72.86 +/- 0.40
Mean Validation Accuracy: 70.44 +/- 1.29


In [14]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(100, 
                        input_shape = (11, 72), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 100,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 0s 193us/step
(Training, Validation) accuracies: (73.14,71.38)
2516/2516 [==============================] - 1s 206us/step
(Training, Validation) accuracies: (72.77,72.30)
2515/2515 [==============================] - 1s 217us/step
(Training, Validation) accuracies: (73.81,70.89)
2515/2515 [==============================] - 1s 230us/step
(Training, Validation) accuracies: (73.19,70.74)
2514/2514 [==============================] - 1s 243us/step
(Training, Validation) accuracies: (73.69,67.70)
Mean Training Accuracy: 73.32 +/- 0.38
Mean Validation Accuracy: 70.60 +/- 1.55


In [15]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(150, 
                        input_shape = (11, 72), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 150,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 1s 259us/step
(Training, Validation) accuracies: (75.69,70.03)
2516/2516 [==============================] - 1s 267us/step
(Training, Validation) accuracies: (74.56,71.62)
2515/2515 [==============================] - 1s 280us/step
(Training, Validation) accuracies: (75.89,69.70)
2515/2515 [==============================] - 1s 296us/step
(Training, Validation) accuracies: (74.35,70.06)
2514/2514 [==============================] - 1s 317us/step
(Training, Validation) accuracies: (75.72,67.74)
Mean Training Accuracy: 75.24 +/- 0.65
Mean Validation Accuracy: 69.83 +/- 1.24


In [16]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(150, 
                        input_shape = (11, 72), 
                        dropout = 0.75))
    lstm_model.add(Dense(units = 100,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 200, 32, False)

2516/2516 [==============================] - 1s 319us/step
(Training, Validation) accuracies: (74.48,70.87)
2516/2516 [==============================] - 1s 326us/step
(Training, Validation) accuracies: (74.68,72.02)
2515/2515 [==============================] - 1s 350us/step
(Training, Validation) accuracies: (74.89,70.14)
2515/2515 [==============================] - 1s 358us/step
(Training, Validation) accuracies: (73.99,70.30)
2514/2514 [==============================] - 1s 387us/step
(Training, Validation) accuracies: (75.86,68.02)
Mean Training Accuracy: 74.78 +/- 0.62
Mean Validation Accuracy: 70.27 +/- 1.30
